In [0]:
storageName = dbutils.widgets.get("storageName")
storageKey = dbutils.widgets.get("storageKey")
fileName = dbutils.widgets.get("fileName")

spark.conf.set(
  f"fs.azure.account.key.{storageName}.blob.core.windows.net",
  storageKey)

In [0]:
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ";"

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(f"wasbs://data@{storageName}.blob.core.windows.net/{fileName}")

display(df)

In [0]:
df2 = df.groupBy("Item Type") \
  .avg("Total Profit") \
  .dropna() \
  .withColumnRenamed("Item Type", "ItemType") \
  .withColumnRenamed("avg(Total Profit)", "AvgTotalProfit")
display(df2)

In [0]:
permanent_table_name = "sales_data_agg"
df2.write.format("parquet").saveAsTable(permanent_table_name, mode = "append")